## Non-Negative Matrix Factorization for Exploring the European Parliament's Topic Agenda
### Assignment 2 for Machine Learning Complements class
By Alexandra de Carvalho, Luís Costa, Nuno Pedrosa

#### Importing the needed Python libraries
We will use Pandas for dataframe manipulation.

In [52]:
import os
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import pandas as pd

# for modeling 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

# for text processing
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /home/alexa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alexa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/alexa/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alexa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#### Importing the data

In [9]:
# expand pandas df column display width to enable easy inspection
pd.set_option('max_colwidth', 150)

# read the textfiles to a dataframe
dir_path = 'sample' # folder path
files = [] # list to store files

for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(os.path.join(dir_path, path))
    else:
        subpath = os.path.join(dir_path, path)
        for path2 in os.listdir(subpath):
            if os.path.isfile(os.path.join(subpath, path2)):
                files.append(os.path.join(subpath, path2))

#### Tokenizing
To make all of the text in the speeches as comparable as possible we need to remove punctuation, capitalization, numbers, and strange characters.

In [65]:
text_tokens = dict()
for filename in files:
    with open(filename, 'rb') as f:
        lines = f.readlines()
    
        for line in lines:
            text_tokens[filename] = text_tokens.get(filename, set()) | set([token.lower() for token in re.split('\W+', str(line)) if len(token) > 3 and not token.isnumeric() and not token in stopwords.words('english')])

#### Lemmatizing

In [68]:
wordnet_lemmatizer = WordNetLemmatizer()   # stored function to lemmatize each word
is_noun = lambda pos: pos[:2] == 'NN'

text_tokens = {doc:set([wordnet_lemmatizer.lemmatize(word) for (word, pos) in pos_tag(token) if is_noun(pos)]) for doc, token in text_tokens.items()} # list comprehension to lemmatize all words